In [16]:
from IPython.core.display import HTML

def load_css():
    styles = open("Slog_zvezka.css", "r").read()
    return HTML("<style>{}</style>".format(styles))

load_css()


<h1>Analiza fantazijskih knjig</h1>
<div class='Podnaslov'>Projektna naloga pri predmetu Uvod v programiranje</div>